In [5]:
import argparse
import logging
import os
import time
import shutil
import numpy as np
import pandas as pd
import sys
import pickle
sys.path.append('/home/ubuntu/BayesCard')
from DataPrepare.join_data_preparation import prepare_sample_hdf
from DataPrepare.prepare_single_tables import prepare_all_tables
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.schemas import gen_DB0_schema

In [2]:
table_csv_path = '/home/ubuntu/data_CE/imdb/{}.csv'
schema = gen_job_light_imdb_schema(table_csv_path)

In [4]:
df_rows = pd.read_csv('/home/ubuntu/data_CE/imdb/title.csv', header=None, escapechar='\\', encoding='utf-8',
                              quotechar='"',
                              sep=",")

/home/ubuntu/miniconda3/envs/skyscraper/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3229: DtypeWarning: Columns (5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_rows.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,80889,(#1.66),NaN,7,1980.0,NaN,NaN,80815.0,1.0,66.0,NaN,4de45f35edf0b753c54ab72dcbe68bb5
1,5156,Josie Duggar's 1st Shoes,NaN,7,2010.0,NaN,J2326,5022.0,4.0,13.0,NaN,8d492850166137b550ceb0e9a5b46086
2,197772,(#2.8),NaN,7,1962.0,NaN,NaN,197751.0,2.0,8.0,NaN,152b0f7213dd467760823b0d6387a6ec
3,111913,(2012-09-13),NaN,7,2012.0,NaN,NaN,111095.0,NaN,NaN,NaN,daf144630ef1ab1cd5d1d028b406cdcf
4,117556,(#1.1042),NaN,7,NaN,NaN,NaN,117506.0,1.0,1042.0,NaN,be62adfea75e456d47ddab5c68ac96b0


In [7]:
len(df_rows.columns)

12

In [10]:
sys.path.append("/home/ubuntu/BayesCard")
schema = gen_job_light_imdb_schema('/home/ubuntu/data_CE/imdb/{}.csv')
ensemble_location = "/home/ubuntu/BayesCard/Benchmark/IMDB/"
from DataPrepare.query_prepare_BayesCard import prepare_join_queries
query_filename = "/home/ubuntu/BayesCard/Benchmark/IMDB/job-light.sql"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path=None, 
                                            query_filename=query_filename, true_card_exist=True)

In [7]:
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path=ensemble_location):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns.append(bn)
            #bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)
#with open(query_filename, "rb") as f:
 #   real_query = f.readlines()
#queries = bn_ensemble.parse_query_all(parsed_queries)

3_chow-liu_1.pkl
4_chow-liu_1.pkl
2_chow-liu_1.pkl
0_chow-liu_1.pkl
1_chow-liu_1.pkl


In [11]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()
queries = bn_ensemble.parse_query_all(parsed_queries)

In [12]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2||715\n' 

true cardinality 715, predicted 3536.0838012929016 with q-error 4.945571750060003
predicting query no 1: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010||9\n' 

true cardinality 9, predicted 19.90215698738552 with q-error 2.211350776376169
predicting query no 2: b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2 AND t.production_year>2010||47\n' 

true cardinality 47, predicted 161.0907814841737 with q-error 3.427463435833483
predicting query no 3: b'SELECT COUNT(*) FROM movie_companies m

predicting query no 37: b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mk.movie_id AND t.id=mi_idx.movie_id AND t.production_year>2010 AND t.kind_id=1 AND mi.info_type_id=8 AND mi_idx.info_type_id=101||150780\n' 

true cardinality 150780, predicted 978596.7461906449 with q-error 6.490229116531668
predicting query no 38: b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mk.movie_id AND t.id=mi_idx.movie_id AND t.kind_id=1 AND mi.info_type_id=8 AND mi_idx.info_type_id=101||3243247\n' 

true cardinality 3243247, predicted 15118380.63300502 with q-error 4.6614952956111635
predicting query no 39: b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mk.movie_id AND t.id=mi_idx.movie_id AND t.production_year>2005 AND mi.info_type_id=8 AND mi_idx.info_type_id=101||1043763\n' 

true cardinality 10437

predicting query no 68: b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mc.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND ci.role_id=2 AND mi.info_type_id=16 AND t.production_year>2000 AND t.production_year<2005 AND mk.keyword_id=7084||268172\n' 

true cardinality 268172, predicted 3990775.2848841464 with q-error 14.881401805125615
predicting query no 69: b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100 AND t.production_year>200||258385218\n' 

true cardinality 258385218, predicted 2314516371.247255 with q-error 8.957619128379298
q-error 50% percentile is [1.57221377]
q-error 90% percentile is 6.5824382316996415
q-error 95% percentile is 8.831415853801895
q-error 99% percentile is 15.939205786780047
q-error 100% perc

/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
